In [1]:
import tensorflow as tf

In [5]:
tf.InteractiveSession()

# Variables

상수의 경우에는 tf.constant인데 변수의 경우에는 tf.Variable()이다. 

* tf.constant = op
* tf.Variable = Class

In [6]:
a = tf.Variable(2, name='scalar')
b = tf.Variable([2,3], name='vector')
c = tf.Variable([[0,1], [2,3]], name='matrix')

W = tf.Variable(tf.zeros([784,10]))

In [29]:
# print a.eval(), b.eval(), c.eval(), W.eval() # error. 이유는 바로 아래에.

x = tf.Variable(...) 은 아래와 같은 일련의 op 과정을 포함하는 것이다. 

* x.initializer # init op
* x.value() # read op
* x.assign(...) # write op
* x.assign_add(...) # and more

Variables는 사용하기 전에 반드시 초기화해 주어야 한다. 초기화해 주지 않고 바로 사용하면 위의 예에서와 같이 에러가 발생하는 것이다. 

### Variables의 초기화 방법

1. 모든 변수들 한꺼번에 초기화하기
2. 모든 변수들 중 일부만 초기화하기
3. 모든 변수등 중 하나의 변수만 초기화하기

### 1. 모든 변수들 한꺼번에 초기화하기

In [24]:
a = tf.Variable(2, name='scalar')
b = tf.Variable([2,3], name='vector')
c = tf.Variable([[0,1], [2,3]], name='matrix')

In [25]:
init = tf.global_variables_initializer() # 여기에서 초기화된다.
with tf.Session() as sess:
    sess.run(init)

### 2. 모든 변수들 중 일부만 초기화하기 

In [26]:
init_ab = tf.variables_initializer([a,b], name="init_ab") # 여기에서 초기화된다
with tf.Session() as sess:
    sess.run(init_ab)

### 3.  모든 변수들 중 하나의 변수만 초기화하기

초기화하지 않고 sess.run(W) 하면 에러!

In [27]:
W = tf.Variable(tf.zeros([784,10])) # 여기에서 초기화되지 않는다
with tf.Session() as sess:
    sess.run(W.initializer) # 여기에서 초기화해 준다. 

eval()은 변수이다.

In [32]:
W = tf.Variable(tf.truncated_normal([5,4]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print W

Tensor("Variable_7/read:0", shape=(5, 4), dtype=float32)


In [39]:
with tf.Session() as sess:
    sess.run(W.initializer)
    print W.eval() 

[[-1.12916279 -0.67341095 -1.19955444 -1.50376272]
 [-1.65785253  1.17467988  0.25464714 -0.76969141]
 [-0.79108399 -0.29529995 -1.94731283  1.780285  ]
 [ 1.1511066  -0.52168369 -1.23300922  0.12727532]
 [-0.85992777  0.30541098  0.75430328 -0.66003859]]


위에서 print W.eval() 은 print tf.get_default_session().run(W) 와 동일하므로, 아래와 같이 다시 써 줄 수 있다.

In [42]:
with tf.Session() as sess:
    sess.run(W.initializer)
    print tf.get_default_session().run(W) # W는 위에서 초기화했으므로 다시 할 필요X

[[-1.37914598 -0.67535728 -1.49651134 -0.36050206]
 [-0.77418554 -0.55871153 -0.16459692 -0.94768941]
 [ 0.88195693 -0.45734224  1.23663425  0.35517156]
 [ 0.15902209  0.93294448  1.0256412  -0.212182  ]
 [ 0.29597327 -0.54866135  1.07082534 -0.49731308]]


### tf.Variable.assign()

In [43]:
W = tf.Variable(10)
W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    print W.eval()

10


위에서 W.assign(100) 해 준 것은 그저 assign op을 생성만 했을 뿐 run을 안 해 줬기 때문에 효과가 없게 되는 것이다. 아래와 같이 해 줘야 한다.

In [44]:
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    sess.run(assign_op)
    print W.eval()

100


사실 assign op은 초기화도 함께 해 준다. 다시 말해 initializer op은 그 변수의 초기화 값을 변수에 할당하는 assign op 인 것이다. 그러므로 sess.run(W.initializer) 는 생략 가능하다. 아래의 예에서 이를 보도록 하자.

In [46]:
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(assign_op)
    print W.eval()

100


아래의 예를 보고 생각해 보자.

In [49]:
my_var = tf.Variable(2, name='my_var')
my_var_times_two = my_var.assign(2 * my_var)
with tf.Session() as sess:
    print sess.run(my_var.initializer)
    print sess.run(my_var)
    print sess.run(my_var_times_two)

None
2
4


my_var는 sess.run() 에서 초기화가 되어야 하는데 반해 my_var_times_two는 my_var.assign() 이 되었기 때문에 이미 초기화되어 있는 것이다.

그리고 주의해야 할 것은 my_var_times_two = my_var.assign(2 * my_var) 에서 = 는 할당의 의미가 아니며, my_Var_times_two는 변수가 아니라 op 이라는 점이다!

그렇다면 아래와 같은 경우에는 어떻게 나올 지 생각해 보자. 

In [51]:
with tf.Session() as sess:
    print sess.run(my_var.initializer)
    print sess.run(my_var)
    print sess.run(my_var_times_two)
    print sess.run(my_var_times_two)
    print sess.run(my_var_times_two)

None
2
4
8
16


my_var_times_two가 변수가 아니라 op이기 때문에 my_var_times_two 가 fetch될 때마다 2 * my_var를 assign하게 되는 것이다!

이번에는 assign_add()와 assign_sub()를 살펴 보자. assign_add()와 assign_sub() 와 같은 op은 my_var의 원래의 값을 필요로 하기 때문에 my_var 변수를 초기화하지 못 한다. 

In [55]:
my_var = tf.Variable(10)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    print my_var.eval() # 10
    sess.run(my_var.assign_add(10))
    print my_var.eval() # 20
    sess.run(my_var.assign_sub(2))
    print my_var.eval() # 18

10
20
18


각 세션은 변수에 대해 자기 자신만의 copy를 유지한다.

In [59]:
W = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
# print W.eval() # error. Why?
sess2.run(W.initializer)
# print W.eval() # error. Why?

print sess1.run(W.assign_add(10)) # 20
print sess2.run(W.assign_sub(2)) # 8

20
8


In [60]:
print sess1.run(W.assign_add(100)) # 120
print sess2.run(W.assign_sub(50)) # -42

120
-42


### 기존에 있던 변수를 이용해서 새로운 변수 초기화해 주기

사실 아래와 같이 하는 것이 매우 일반적으로 사용되는 방식이기는 한데, 기존 변수가 초기화되어 있지 않을 수도 있기 때문에 안전한 방식은 아니다

In [2]:
W = tf.Variable(tf.truncated_normal([700,10]))
U = tf.Variable(2*W)

그래서 이런 경우에는 initialized_value()라는 함수를 이용해 주는 것이 보다 안전하다.

In [4]:
W = tf.Variable(tf.truncated_normal([700,10]))
U = tf.Variable(2*W.initialized_value()) # initialized_value() 함수 이용!

예제 1

In [8]:
sess = tf.InteractiveSession()
w = tf.Variable(2)
u = tf.Variable(2*w)
sess.run(w.initializer)
print w.eval()
# print u.eval() # error - Attempting to use uninitialized value Variable_9

2


예제 2

In [12]:
w = tf.Variable(2)
u = tf.Variable(2*w)
sess.run(w.initializer)
sess.run(u.initializer)
print w.eval()
print u.eval()

2
4


예제 3

In [13]:
w = tf.Variable(2)
u = tf.Variable(2*w.initialized_value())
sess.run(u.initializer)
print u.eval()

4


### Session vs. InteractiveSession

Session과 InteractiveSession의 차이는 : InteractiveSession은 자기 자신을 디폴트로 만든다는 것이다.

In [15]:
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
print c.eval() # 이런 식으로 sess 지정 없이 바로 c.eval() 사용할 수 있다

30.0


c.eval()은 tf.get_default_session().run(c) 와 동일하다!

In [18]:
print tf.get_default_session().run(c)
sess.close()

30.0


### Control dependencies

op을 실행할 때 기존에 미리 실행되어 있어야 할 op들을 지정해 줄 수 있습니다.

예를 들어 transfer learning에서 가져온 가중치.. op 들이 확실히 로딩되었는지 확인하고 다음 op을 진행해야 하는 경우에 유용할 수 있습니다. 

In [23]:
g = tf.get_default_graph()
with g.control_dependencies([a,b,c]):
    d = tf.constant(1) # d와 e는 a,b,c가 실행된 다음에만 실행될 수 있다. 
    e = tf.constant(1)

아래와 같이 nest 처리도 가능합니다.

In [34]:
with g.control_dependencies([a, b]):
    e = tf.constant(1) # a,b 실행이 전제되어야만 동작한다
    with g.control_dependencies([c, d]):
        f = tf.constant(1) # a,b,c,d 실행이 전제되어야만 동작한다

중간에 clear해 주려면 인자로 None을 주면 됩니다. 

In [35]:
with g.control_dependencies([a, b]):
    e = tf.constant(1) # a,b 실행이 전제되어야만 동작한다
    with g.control_dependencies(None):
        f = tf.constant(1) # a,b 실행 전제하지 않고 그냥 정상적으로 동작한다
        with g.control_dependencies([c, d]):
              g = tf.constant(1) # a,b 실행 전제는 없고, c,d 실행이 전제되어야만 동작한다

The control dependencies context applies only to ops that are constructed within the context. Merely using an op or tensor in the context does not add a control dependency. The following example illustrates this point:

In [36]:
# WRONG
def my_func(pred, tensor):
  t = tf.matmul(tensor, tensor)
  with tf.control_dependencies([pred]):
    # The matmul op is created outside the context, so no control
    # dependency will be added.
    return t

# RIGHT
def my_func(pred, tensor):
  with tf.control_dependencies([pred]):
    # The matmul op is created in the context, so a control dependency
    # will be added.
    return tf.matmul(tensor, tensor)